In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bertopic[visualization]


     |█████▊                          | 10 kB 18.9 MB/s eta 0:00:01
     |███████████▍                    | 20 kB 11.2 MB/s eta 0:00:01
     |█████████████████               | 30 kB 9.1 MB/s eta 0:00:01
     |██████████████████████▊         | 40 kB 8.3 MB/s eta 0:00:01
     |████████████████████████████▍   | 51 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 2.7 MB/s 
     |████████████████████████████████| 6.4 MB 9.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 26.5 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
     |████████████████████████████████| 3.4 MB 47.6 MB/s 
     |████████████████████████████████| 6.8 MB 28.4 MB/s 
     |████████████████████████████████| 1

In [1]:
# Colab może mieć problemy z czytaniem HDFów. Konieczny jest upgrade tables
!pip install --upgrade tables

     |████████████████████████████████| 5.9 MB 3.9 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [5]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import gensim
from gensim.models import FastText, KeyedVectors
import hdbscan
import umap

In [6]:
# Ścieżka do katalogu z notebookami na Colabie
path = '../content/drive/MyDrive/Colab Notebooks/'

In [7]:
df = pd.read_hdf(path+'outputs/budzety_clean.hd5', key='df_all')

In [8]:
df = df.reset_index()
del df['index']
df

,rok,miasto,nazwa,WSZYSTKO,tokens,zbiornik,sadzić,estetyczny,lęgowy,ekologia,rekultywacja,trawnik,nasadzić,bylina,łąka,park,ogrodniczy,ogród,ozdobny,roślina,kwietny,ekologiczny,zazielenić,parkowy,leśny,natura,las,kwiatowy,roslin,zasadzić,gaj,kwiat,krzew,drzewo,trawiasty,ptasi,sztuczny,zielony,przyroda,przyrodniczy,żywopłot,dębowy,kasztanowiec,leś,powietrze,dziki,naturalny,trawniki,antysmogowy,estetyka,kwietnik,deszczowy,kwitnąć,ogrodowy,pielęgnacja,smog,zieleniec,names,szkola,ulica,oswietlenie
0,2020,GDYNIA,Projektistatus,0,[projektistatus],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,projektistatus,False,False,False
1,2020,GDYNIA,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,3,"[doposażenie, park, pomiędzy, estakada, kwiatk...",False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,doposażenie park pomiędzy estakada kwiatkowski...,False,True,False
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy O...,0,"[cykliczny, warsztat, senior, dzielnica, obłuż...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,cykliczny warsztat senior dzielnica obłuże roz...,False,False,False
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",3,"[ogród, sąsiedzki, dziki, sadź, ul, źródlana]",False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,ogród sąsiedzki dziki sadź ul źródlana,False,True,False
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy...,0,"[montaż, słup, oświetleniowy, chodnik, ul, nis...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,montaż słup oświetleniowy chodnik ul niski odc...,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23025,2020,KRAKÓW,Zielony skwer na Zabtociu,1,"[zielony, skwer, zabtociu]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,zielony skwer zabtociu,False,False,False
23026,2020,KRAKÓW,Znani Biatopradniczanie dla Mieszkanców,0,"[znany, mieszkanców]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,znany mieszkanców,False,False,False
23027,2020,KRAKÓW,ZRÓBMY SKWER 

In [9]:
docs = list(df.loc[:, 'names'].values)

docs = [str(elem) for elem in docs] 

In [13]:
model = KeyedVectors.load_word2vec_format(path+'fasttext_ekolo.bin', binary=True)

In [14]:
def get_X_fromFastt(model, tokens):
    def getFasttext_rep(words):
        return np.mean([model.wv[word] for word in words if word in model.wv], axis=0)
    
    X = tokens.map(getFasttext_rep)
    def_vec = X[False == X.isnull()].mean()
    return np.stack(X.map(lambda x: def_vec if str(x) == 'nan' else x))

In [15]:
X = get_X_fromFastt(model, df['tokens'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:
X

array([[-0.2624801 , -2.1912763 , -0.79641235, ...,  1.4660714 ,
         1.6038737 ,  1.254093  ],
       [ 0.11196489, -0.76484144, -4.2596626 , ...,  2.1365523 ,
        -1.8156078 ,  2.1604927 ],
       [-1.4361684 , -1.7526237 , -2.9965186 , ...,  1.0898724 ,
        -0.890457  ,  1.4475201 ],
       ...,
       [-0.52840495, -1.0764297 , -3.5100257 , ...,  1.6179681 ,
         0.38137087,  2.9668038 ],
       [-3.1775055 , -1.5508636 , -4.6909466 , ...,  0.55942774,
        -0.6343127 ,  4.635996  ],
       [ 1.3956118 , -2.6173391 , -3.006124  , ...,  1.2205263 ,
        -0.7016969 ,  2.3971343 ]], dtype=float32)

In [68]:
umap_model = umap.UMAP(n_neighbors=15,
                       n_components=10,
                       min_dist=0.0,
                       metric='cosine',
                       low_memory=False)

# HDBSAN - klasteryzacja dokumentów
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True)

model_bert = BERTopic(top_n_words=30,
                       calculate_probabilities=True,
                       verbose=True,
                       language='polish')

topics, probs = model_bert.fit_transform(docs, X)


2021-12-30 16:29:41,769 - BERTopic - Reduced dimensionality with UMAP
2021-12-30 16:29:43,481 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [69]:
model_bert

In [70]:
# !!!!!! WAŻNE !!!!!!
#
# Jeżeli masz w  folderze models pliki:
# 'main_bert_topic_fasttext.model' i 'main_bert__fasttext_topics_probs.npy' to nie uruchamiaj
# tej komórki 

model_bert.save(path+'models/main_bert_topic_fasttext_from_global.model')
with open(path+'models/main_bert_fasttext_from_global_topics_probs.npy', 'wb') as f:
    np.save(f, topics)
    np.save(f, probs)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [71]:
# Wczytanie zapisanych modeli
model_bert = BERTopic.load(path+'models/main_bert_topic_fasttext_from_global.model')
with open(path+'models/main_bert_fasttext_from_global_topics_probs.npy', 'rb') as f:
    topics = np.load(f)
    probs = np.load(f)

In [72]:
model_bert.get_representative_docs()

{0: ['rewitalizacja park kleparski',
  'street workout park dygata',
  'park aktywność park czechówek'],
 1: ['utwardzenie trawnik parkować samochód przebudowa dojazd ulica kąkolowy strona ul',
  'stworzenie zatoka postojowy strefa kiss ride sp remonta chodnik ul chabrowy borowikowy bzowy',
  'naprawa nawierzchnia droga osiedlowy utworzenie miejsce parkingowy wykorzystywać zniszczyć teren nieużytek ul emilii plater'],
 2: ['utwardzenie ulica ciechociński',
  'utwardzenie ulica ostrygowy',
  'remonta chodnik ulica szczęśliwicki'],
 3: ['rozbudowa plac zabawa ul gojawiczyńskiej',
  'żabiniec doposażenie plac zabawa ul konecznego',
  'rozbudowa plac zabawa ul mazańcowickiej'],
 4: ['modernizacja pracownia komputerowy szkoła podstawowy ul broniewskiego',
  'kompleks sportowo rekreacyjny szkoła podstawowy ul zajchowskiego',
  'budowa nowy ogrodzenie obiekt boisko obiekt sportowy ul nowobielańskiej'],
 5: ['poprawa bezpieczeństwo mieszkaniec gmina mieroszów powiat wałbrzyski poprzez doposaże

In [73]:
# Sprawdzenie tematów
model_bert.get_topics()

{-1: [('ul', 0.0028646991296163754),
  ('dziecko', 0.002577160315917204),
  ('ulica', 0.002520258063815957),
  ('wykonanie', 0.0024992243202574104),
  ('rowerowy', 0.0024443118964554045),
  ('budowa', 0.0023714025714885304),
  ('miejsce', 0.00235089545944093),
  ('chodnik', 0.0023479950206249755),
  ('teren', 0.0023106486414142292),
  ('warsztat', 0.0022454655115441303),
  ('zieleń', 0.002244124836259236),
  ('skwer', 0.0022420294610454258),
  ('osiedle', 0.0022418800476164437),
  ('parking', 0.0022216852792031967),
  ('mieszkaniec', 0.0022122202690111016),
  ('zielony', 0.002191387332147381),
  ('plac', 0.002166153354537361),
  ('boisko', 0.002161123351406936),
  ('pieszy', 0.0021040968084024216),
  ('rewitalizacja', 0.0021007075710916426),
  ('budynek', 0.00209342914892806),
  ('miejski', 0.0020843379894611683),
  ('biblioteka', 0.002082891669435766),
  ('nawierzchnia', 0.002079360143367054),
  ('wraz', 0.0020758183762105086),
  ('senior', 0.002074278729596431),
  ('wzdłuż', 0.002058

In [74]:
# W domyślnym modelu mamy 800 tematów
# ograniczmy ich ilość
N_TOPICS = 20

In [75]:
# Redukcja tematów 
new_topics, new_probs = model_bert.reduce_topics(docs, topics, probabilities=probs,
 nr_topics=N_TOPICS)

# Zapiszmy nowe tematy
# with open(path+'models/new_bert_topics_probs.npy', 'wb') as f:
#     np.save(f, new_topics)
#     np.save(f, new_probs)

2021-12-30 16:30:04,172 - BERTopic - Reduced number of topics from 360 to 21


In [77]:
# Podgląd tematów
model_bert.get_topic_info()

,Topic,Count,Name
0,-1,15514,-1_ul_dziecko_zielony_ulica
1,0,1064,0_park_street_tężnia_workout
2,1,803,1_droga_ul_chodnik_wraz
3,2,667,2_plac_zabawa_pm_przedszkole
4,3,533,3_gra_rodzinny_mapa_terenowy
5,4,424,4_pieszy_bezpieczny_przejście_próg
6,5,382,5_ulica_remonta_chodnik_nawierzchnia
7,6,356,6_sp_pracownia_xxi_szatnia
8,7,356,7_remonta_chodnik_ul_budowa
9,8,322,8_szkoła_podstawowy_zespół_boisko


In [26]:
topic_dict = dict()
for n in range(-1, N_TOPICS):
  topic_dict[n] = [name for name, prob in model_bert.get_topic(n)]
  print(f'{n} {topic_dict[n]}', sep='\n')

-1 ['ul', 'plac', 'budowa', 'zabawa', 'ulica', 'plac zabawa', 'park', 'dziecko', 'zielony', 'chodnik', 'teren', 'remonta', 'miejsce', 'osiedle', 'rowerowy', 'droga', 'modernizacja', 'skwer', 'mieszkaniec', 'wraz']
0 ['szkoła', 'boisko', 'podstawowy', 'szkoła podstawowy', 'sp', 'sportowy', 'zespół', 'budowa boisko', 'budowa', 'modernizacja', 'zespół szkoła', 'szkolny', 'boisko sportowy', 'teren', 'lo', 'modernizacja boisko', 'ul', 'plac', 'zabawa szkoła', 'kompleks']
1 ['zajęcie', 'dziecko', 'młodzież', 'dziecko młodzież', 'warsztat', 'samoobrona', 'kobieta', 'ruchowy', 'sportowy', 'kurs', 'samoobrona kobieta', 'dorosły', 'bezpłatny', 'bezpłatny zajęcie', 'senior', 'elementy samoobrona', 'zajęcie fizyczny', 'poprawiać koordynacja', 'koordynacja ruchowy', 'fizyczny elementy']
2 ['pies', 'wybieg', 'wybieg pies', 'park', 'zwierzę', 'psi', 'bezdomność zwierzę', 'bezdomność', 'zwierzę teren', 'teren osiedle', 'kota', 'kot', 'żyć', 'ograniczanie', 'ograniczanie bezdomność', 'psi park', 'wolno

In [78]:
# Dalsze kroki są takie same jak w poprzednich notatnikach.